In [483]:
has_commodity_order = ["ARKH3a", "ARKH3b", "HT101", "HT102", "HT106", "HT114a", "HT116a",
                       "HT116b", "HT12", "HT121", "HT125a", "HT125b", "HT128a", "HT129",
                       "HT130", "HT131a", "HT131b", "HT137", "HT139", "HT14", "HT18", "HT2",
                       "HT21", "HT23a", "HT23b", "HT27a", "HT28a", "HT28b", "HT30", "HT32",
                       "HT33", "HT34", "HT44a", "HT50a", "HT58", "HT91", "HT94a", "HT96b",
                       "KH11", "KH12", "KH21", "KH5", "KH55", "KH9", "KNZb35", "TY3a", "ZA11a",
                       "ZA18a", "ZA6b", "HT35", "HT90", "HT100", "HT99a"]


In [484]:
bronze = ["𐙈"]
grain = ["𐙉", "𐛵", "𐛟","𐛬","𐛭","𐛮","𐛯","𐛱","𐛲","𐜠𐛱","𐙗","𐜙","𐜚","𐜛"]
oilProducts = ["𐙖", "𐙜", "𐜉","𐜊","𐜋","𐜌","𐜍","𐜎","𐜏","𐜐","𐜑","𐜒","𐜓","𐘸","𐜔","𐜕","𐜖","𐜗","𐜘"]
corianderProducts = ["𐙘"]
olive = ["𐙋", "𐛺"]
figs = ["𐘝"]
vineProducts = ["𐙍","𐛾","𐛽","𐘞𐙍","𐛼","𐜀","𐜂", "𐛻","𐛿","𐜁","𐘋𐙍","𐙍𐘻"]
QAProducts = ["𐚻","𐚹","𐚼","𐚺"]
EProducts = ["𐘡", "𐛆","𐛅"]
MIProducts = ["𐘻","𐛚","𐛜","𐛛","𐛙","𐛝"]
livestock = ["𐘖", "𐛂"]

commodity_order = [bronze, grain, oilProducts, corianderProducts, olive,
                   figs, vineProducts, QAProducts, EProducts, MIProducts, livestock]

heading = ("inscription, recipient, bronze, grain, oil, coriander, olive,"
            " figs, wine, QA, E, MI, livestock, uncategorized")


In [485]:
import json

json_file = open('CommodityOrderData/150-inscriptions.js')
inscriptions = json.load(json_file)


findspots = {}
sites = {}
for inscription in inscriptions:
    findspots[inscription["name"]] = inscription["findspot"]
    sites[inscription["name"]] = inscription["site"]
    


In [486]:
import os
inscriptions = []

inputdir = "CommodityOrderData/transaction_data/"
for subdir, dirs, files in os.walk(inputdir):
    for file in files:
        if file[-2:] != "js":
            continue
        json_file = open(subdir + os.sep + file)
        inscription = json.load(json_file)
        inscriptions.append(inscription)


flatten = lambda l: [item for sublist in l for item in sublist]


In [487]:
test_items = ["","","1:𐙉" ,"2:𐜗 3:𐜉 4:𐙘" ,"5:𐙋" ,"","","","",""]
def violatesOrder(items):
    order_so_far = []
    for item in items:
        order = list(filter(lambda x: x != "", [it.split(':')[0] for it in item.split(' ')]))
        if not order:
            continue
        for x in order:
            for y in order_so_far:
                if x < y:
                    return True
            order_so_far.append(x)
    return False
print(violatesOrder(test_items))

False


In [488]:

def writeCommodityLine(output, recipient, no_of_commodity_groups, inscription):
    if not flatten(output):
        return
    
    uncertain = "commodityOrder" in inscription and inscription["commodityOrder"] == "uncertain"
    violation = "!" if violatesOrder(output[:-1]) else ""
    violation += "+" if uncertain else ""
    
    output_lines.append(inscription["name"] + violation + ',' + recipient + ',' + ','.join(output))
    no_of_commodity_groups.append(len(output) - output.count(""))
    
def collectCommodityOrder(word_infos, inscription):
    output_lines = []
    no_of_commodity_groups = []
    output = [""] * (len(commodity_order) + 1)
    order = 1
    recipient = ""
    for index, word_data in enumerate(word_infos):
        if "description" not in word_data:
            continue
        if word_data["description"] == "recipient":
            if recipient != "":
                writeCommodityLine(output, recipient, no_of_commodity_groups, inscription)
                output = [""] * (len(commodity_order) + 1)
                order = 1
            recipient = word_data["transliteratedWord"]
            continue
        if word_data["description"] != "commodity":
            continue
        word = word_data["word"].replace('\U0001076b', '')
        found_commodity = False
        for index, commodity in enumerate(commodity_order):
            if word not in commodity:
                continue
            output[index] += str(order) + ":" + word + " "
            order += 1
            found_commodity = True
        if not found_commodity:
            output[len(commodity_order)] += str(order) + ":" + word + " "

    writeCommodityLine(output, recipient, no_of_commodity_groups, inscription)

    return (output_lines, no_of_commodity_groups)

def collectCommodityOrderImproved(word_infos, inscription):
    output_lines = []
    no_of_commodity_groups = []
    output = [""] * (len(commodity_order) + 1)
    order = 1
    prev_tran_id = ""
    tran_id = ""
    recipients = {}
    
    for index, word_data in enumerate(word_infos):
        if "description" not in word_data:
            continue
        if "transactionID" in word_data:
            tran_id = word_data["transactionID"]
        
        if word_data["description"] == "recipient":
            recipients[tran_id] = word_data["transliteratedWord"]
            continue
        if tran_id != prev_tran_id:
            writeCommodityLine(output, recipients[prev_tran_id], no_of_commodity_groups, inscription)
            output = [""] * (len(commodity_order) + 1)
            order = 1
        if word_data["description"] != "commodity":
            continue
        word = word_data["word"].replace('\U0001076b', '')
        found_commodity = False
        for index, commodity in enumerate(commodity_order):
            if word not in commodity:
                continue
            output[index] += str(order) + ":" + word + " "
            order += 1
            found_commodity = True
        if not found_commodity:
            output[len(commodity_order)] += str(order) + ":" + word + " "
        prev_tran_id = tran_id
    writeCommodityLine(output, recipient, no_of_commodity_groups, inscription)

    return (output_lines, no_of_commodity_groups)



In [489]:
# Commodity order in villa magazine tablets
print(heading)
output_lines = []

no_of_commodity_groups = []
for inscription in inscriptions:
    findspot = findspots[inscription["name"]]
    if findspot != "Villa Magazine":
        continue
    word_infos = inscription["words"]

    commodity_info = collectCommodityOrder(word_infos, inscription)
    output_lines.extend(commodity_info[0])
    no_of_commodity_groups.extend(commodity_info[1])
    
import re
for line in sorted(output_lines, key = lambda x: int(re.sub('[A-Za-z\!\+]', '', x.split(",")[0]))):
    print(line)

print(sum(no_of_commodity_groups) / len(no_of_commodity_groups))

inscription, recipient, bronze, grain, oil, coriander, olive, figs, wine, QA, E, MI, livestock, uncategorized
HT6a,KA-PA,,,,,,1:𐘝 2:𐘝 ,,,,,,2:𐛑 
HT7a,I-RU-JA,,,,,,,,,,,,1:𐙇 
HT8a,JE-DI,,,1:𐜓 ,,,,,,,,,
HT9a,PA-DE,,,,,,,1:𐙍 ,,,,,
HT12,QA-TI-DA-TE,,,1:𐜉 ,2:𐙘 ,,,,,,,,
HT12,QE-TU-NE,,,,,,,,1:𐚻 ,2:𐘡 ,,,
HT12,*560,1:𐙈 ,2:𐜚 ,,,,3:𐘝 ,,4:𐚼 ,,,,5:𐘀𐘚 
HT13,RE-ZA,,,,,,,1:𐙍 ,,,,,
HT14,PU-VIN,,1:𐙉 ,2:𐜗 3:𐜉 ,4:𐙘 ,5:𐙋 ,,,,,,,
HT14,A-PU₂-NA-DU,,1:𐙉 ,2:𐜗 3:𐜉 ,4:𐙘 ,5:𐙋 ,,,,,,,
HT15,KI-RO,,1:𐙉 ,,,,,,,,,,2:𐛹 
HT17,,,,,,,,1:𐙍 ,,,,,
HT18,PA-SE,,1:𐛱 ,2:𐜓 ,3:𐙘 ,,,,,,,,
HT18,SA-RA₂,,1:𐙉 ,,,,2:𐘝 ,,,,,,
HT19,,,,,,,,1:𐙍 ,,,,,
HT21,PI-TA-KA-SE,,1:𐙉 ,2:𐜋 3:𐜎 ,4:𐙘 ,5:𐙋 ,,,,,,,
HT23a!,KA-NA,,1:𐙗 ,2:𐙜 3:𐜍 4:𐜖 5:𐜏 ,,,,7:𐙍 ,8:𐚹 9:𐚺 ,10:𐘡 ,6:𐛚 ,,11:𐘐𐘭𐘹𐘌 11:𐘞𐘞𐘋 11:𐛊 11:𐘺𐘘 
HT23b,NI-RA,,1:𐙗 ,,,,,,2:𐚹 ,3:𐘡 ,,,
HT27a,TI-NI-TA,,,,,,,,,,,,1:𐙇 1:𐙅 1:𐘙 1:𐘚𐘻𐘞𐘴 1:𐘿 
HT27a,KU-*305,,1:𐙗 ,,,,2:𐘝 ,3:𐙍 ,,,,,1:𐘞𐘴𐘆 1:𐙇 
HT27b,MI-DA,,,,,,,,,,,,1:𐘞 1:𐘞 1:𐘙 
HT27b,PA-SE,,,,,,,,,,,,1:𐘞 1:𐘙 1:𐘞 
HT28b,SA-RA₂,,1:𐙉 ,2:𐜉 ,,,3:𐘝 ,4:𐙍 ,,,,,
HT28b,PU

In [490]:
# Commodity order in tablets found outside the HT Villa Magazine
print(heading)
output_lines = []

no_of_commodity_groups = []
for inscription in inscriptions:
    findspot = findspots[inscription["name"]]
    if findspot == "Villa Magazine":
        continue
    if inscription["name"] not in has_commodity_order:
        continue
    word_infos = inscription["words"]

    commodity_info = collectCommodityOrder(word_infos, inscription)
    output_lines.extend(commodity_info[0])
    no_of_commodity_groups.extend(commodity_info[1])

import re
for line in sorted(output_lines, key = lambda x: int(re.sub('[A-Za-z!\+]', '', x.split(",")[0]))):
    print(line)

print(sum(no_of_commodity_groups) / len(no_of_commodity_groups))

inscription, recipient, bronze, grain, oil, coriander, olive, figs, wine, QA, E, MI, livestock, uncategorized
HT2,A-KA-RU,,,1:𐜋 2:𐜊 3:𐜎 ,,,,,,,,,
HT2,KI-RE-TA-NA,,,1:𐜋 2:𐜊 ,,,,,,,,,
ARKH3b,JA-PI,,1:𐙉 ,,,,,,,,,,
ARKH3b,PI-*314,,1:𐙉 ,2:𐙖 ,,,,3:𐙍 ,,,,,
ARKH3a,KA-NE,,,,,1:𐙋 ,,,,,,,2:𐙎 2:𐙩 
ARKH3a,KI-NU,,1:𐙉 ,,,,,,,,,,2:𐘃 2:𐙩 2:𐙢 2:𐙂 
TY3a,ZA-*321,,,1:𐜓 2:𐜋 3:𐜌 4:𐜋 ,,,,,,,,,3:𐘗𐘸 
TY3a,A-DU,,,1:𐜓 2:𐜔 3:𐜕 4:𐜌 ,,,,,,,,,
TY3a,A-DA,,,1:𐜋 2:𐜗 ,,,,,,,,,
TY3a,A-KU-TU-*361,,,1:𐘸 2:𐜖 ,,,,,,,,,
KH5,WI-SA-SA-NE,,1:𐜚 ,,,,,2:𐜂 ,,,,,
KH5,KU-PA-*79,,,,,,1:𐘝 ,,,,,,
ZA6b,I-KU-JU-TI-*28B,,1:𐛭 ,,,,,2:𐛿 3:𐜁 ,,,,,
ZA6b,*312-TA₂,,1:𐛭 ,,,,,,,,,,
KH9,A-SI-SU-PO-A,,,,,,1:𐘝 ,,,,,,
KH9,I-SI,,1:𐜙 2:𐙗 3:𐙗 ,,,,,,,,,,4:𐛌𐙘 
KH11!,A-DU,,1:𐙗 2:𐜚 4:𐙗 ,,,,,3:𐙍 ,,,,,2:𐘲 4:𐙚 5:𐚈𐙗 
KH11,A-TO-*349-TO-I,,1:𐜚 ,,,,2:𐘝 ,3:𐙍 ,,,,,
KH11,A-TA-*350,,,,,,,,,,,,1:𐙕 1:𐙚 1:𐘤𐙗 
ZA11a!,RA-MA-SI,,3:𐛟 ,,,,,1:𐙍 2:𐙍 ,,,,,2:𐘡𐙂𐘘 2:𐘶 
ZA11a,KU-PA,,1:𐛭 ,,,,,,,,,,
ZA11a,PI-*310-A,,,,,1:𐙋 ,,,,,,,
ZA18a,KU-*321,,1:𐛭 ,,,,,,,,,,
ZA18a,B-B,,,,,1:𐙋 ,,,,,,,
KH

In [491]:
# Commodity order in tablets found outside the Haghia Triada
print(heading)
output_lines = []

no_of_commodity_groups = []
for inscription in inscriptions:
    findspot = findspots[inscription["name"]]
    site = sites[inscription["name"]]
    if site == "Haghia Triada":
        continue
    word_infos = inscription["words"]

    commodity_info = collectCommodityOrder(word_infos, inscription)
    output_lines.extend(commodity_info[0])
    no_of_commodity_groups.extend(commodity_info[1])

import re
for line in sorted(output_lines, key = lambda x: x.split(",")[0]):
    print(line)

print(sum(no_of_commodity_groups) / len(no_of_commodity_groups))

inscription, recipient, bronze, grain, oil, coriander, olive, figs, wine, QA, E, MI, livestock, uncategorized
ARKH2,KU-RA,,,,,,,,,,,,1:𐙎 
ARKH3a,KA-NE,,,,,1:𐙋 ,,,,,,,2:𐙎 2:𐙩 
ARKH3a,KI-NU,,1:𐙉 ,,,,,,,,,,2:𐘃 2:𐙩 2:𐙢 2:𐙂 
ARKH3b,JA-PI,,1:𐙉 ,,,,,,,,,,
ARKH3b,PI-*314,,1:𐙉 ,2:𐙖 ,,,,3:𐙍 ,,,,,
KH11,A-TO-*349-TO-I,,1:𐜚 ,,,,2:𐘝 ,3:𐙍 ,,,,,
KH11,A-TA-*350,,,,,,,,,,,,1:𐙕 1:𐙚 1:𐘤𐙗 
KH11!,A-DU,,1:𐙗 2:𐜚 4:𐙗 ,,,,,3:𐙍 ,,,,,2:𐘲 4:𐙚 5:𐚈𐙗 
KH21,*306-QE-DU,,1:𐙗 ,,,,2:𐘝 ,,,,,,
KH4,,,1:𐙗 ,,,,,,,,,,1:𐙚 
KH5,WI-SA-SA-NE,,1:𐜚 ,,,,,2:𐜂 ,,,,,
KH5,KU-PA-*79,,,,,,1:𐘝 ,,,,,,
KH55,,,,1:𐜐 ,,,,,,,,,
KH61!,,,2:𐙉 ,1:𐜐 ,,,,,,,,,
KH8!,,,1:𐜙 3:𐙉 5:𐙗 ,,4:𐙘 ,,2:𐘝 6:𐘝 ,,7:𐚻 ,,,,1:𐘳 7:𐙢 
KH88,*21F-*118,,,,,,1:𐘝 ,,,,,,
KH9,A-SI-SU-PO-A,,,,,,1:𐘝 ,,,,,,
KH9,I-SI,,1:𐜙 2:𐙗 3:𐙗 ,,,,,,,,,,4:𐛌𐙘 
KNZb35,JA-DI-WI,,,1:𐙖 ,,,2:𐘝 ,,,,,,
TY3a,ZA-*321,,,1:𐜓 2:𐜋 3:𐜌 4:𐜋 ,,,,,,,,,3:𐘗𐘸 
TY3a,A-DU,,,1:𐜓 2:𐜔 3:𐜕 4:𐜌 ,,,,,,,,,
TY3a,A-DA,,,1:𐜋 2:𐜗 ,,,,,,,,,
TY3a,A-KU-TU-*361,,,1:𐘸 2:𐜖 ,,,,,,,,,
ZA11a,KU-PA,,1:𐛭 ,,,,,,,,,,
ZA11a,PI-*310-A,,,,,1:𐙋 ,,,,

In [492]:
# Commodity order in tablets found in Haghia Triada
print(heading)
output_lines = []

no_of_commodity_groups = []
for inscription in inscriptions:
    findspot = findspots[inscription["name"]]
    site = sites[inscription["name"]]
    if site != "Haghia Triada":
        continue
    if inscription["name"] not in has_commodity_order:
        continue
    word_infos = inscription["words"]

    commodity_info = collectCommodityOrder(word_infos, inscription)
    output_lines.extend(commodity_info[0])
    no_of_commodity_groups.extend(commodity_info[1])

import re
for line in sorted(output_lines, key = lambda x: int(re.sub('[A-Za-z!\+]', '', x.split(",")[0]))):
    print(line)


print("Number of tablets", len(set([x.split(',')[0] for x in output_lines])))
print("Average number of groups", sum(no_of_commodity_groups) / len(no_of_commodity_groups))

inscription, recipient, bronze, grain, oil, coriander, olive, figs, wine, QA, E, MI, livestock, uncategorized
HT2,A-KA-RU,,,1:𐜋 2:𐜊 3:𐜎 ,,,,,,,,,
HT2,KI-RE-TA-NA,,,1:𐜋 2:𐜊 ,,,,,,,,,
HT12,QA-TI-DA-TE,,,1:𐜉 ,2:𐙘 ,,,,,,,,
HT12,QE-TU-NE,,,,,,,,1:𐚻 ,2:𐘡 ,,,
HT12,*560,1:𐙈 ,2:𐜚 ,,,,3:𐘝 ,,4:𐚼 ,,,,5:𐘀𐘚 
HT14,PU-VIN,,1:𐙉 ,2:𐜗 3:𐜉 ,4:𐙘 ,5:𐙋 ,,,,,,,
HT14,A-PU₂-NA-DU,,1:𐙉 ,2:𐜗 3:𐜉 ,4:𐙘 ,5:𐙋 ,,,,,,,
HT18,PA-SE,,1:𐛱 ,2:𐜓 ,3:𐙘 ,,,,,,,,
HT18,SA-RA₂,,1:𐙉 ,,,,2:𐘝 ,,,,,,
HT21,PI-TA-KA-SE,,1:𐙉 ,2:𐜋 3:𐜎 ,4:𐙘 ,5:𐙋 ,,,,,,,
HT23a!,KA-NA,,1:𐙗 ,2:𐙜 3:𐜍 4:𐜖 5:𐜏 ,,,,7:𐙍 ,8:𐚹 9:𐚺 ,10:𐘡 ,6:𐛚 ,,11:𐘐𐘭𐘹𐘌 11:𐘞𐘞𐘋 11:𐛊 11:𐘺𐘘 
HT23b,NI-RA,,1:𐙗 ,,,,,,2:𐚹 ,3:𐘡 ,,,
HT27a,TI-NI-TA,,,,,,,,,,,,1:𐙇 1:𐙅 1:𐘙 1:𐘚𐘻𐘞𐘴 1:𐘿 
HT27a,KU-*305,,1:𐙗 ,,,,2:𐘝 ,3:𐙍 ,,,,,1:𐘞𐘴𐘆 1:𐙇 
HT28b,SA-RA₂,,1:𐙉 ,2:𐜉 ,,,3:𐘝 ,4:𐙍 ,,,,,
HT28b,PU-RA₂,,,,,,1:𐘝 ,,,,,,
HT28b,JA-*21F,,,,,,,1:𐙍 ,,,,,
HT28b,WI-DI-NA,,,1:𐜉 ,,,,2:𐙍 ,,,,,
HT28a,JA-*21F,,1:𐛱 ,2:𐜋 3:𐜓 4:𐜗 5:𐜖 ,,,,,,,,,
HT28a,SA-RA₂,,,1:𐜉 ,,,2:𐘝 ,3:𐙍 ,,,,,4:𐛧𐙍 
HT28a!,A-RU-DA-RA,,1:𐙉 ,3:𐜉 ,2:𐙘 ,,,,,,,,
HT28

In [493]:
# Commodity order in all tablets
print(heading)
output_lines = []

no_of_commodity_groups = []
for inscription in inscriptions:
    findspot = findspots[inscription["name"]]
    site = sites[inscription["name"]]
    word_infos = inscription["words"]

    commodity_info = collectCommodityOrder(word_infos, inscription)
    output_lines.extend(commodity_info[0])
    no_of_commodity_groups.extend(commodity_info[1])

import re
for line in sorted(output_lines, key = lambda x: x.split(",")[0]):
    print(line)

print(sum(no_of_commodity_groups) / len(no_of_commodity_groups))

inscription, recipient, bronze, grain, oil, coriander, olive, figs, wine, QA, E, MI, livestock, uncategorized
ARKH2,KU-RA,,,,,,,,,,,,1:𐙎 
ARKH3a,KA-NE,,,,,1:𐙋 ,,,,,,,2:𐙎 2:𐙩 
ARKH3a,KI-NU,,1:𐙉 ,,,,,,,,,,2:𐘃 2:𐙩 2:𐙢 2:𐙂 
ARKH3b,JA-PI,,1:𐙉 ,,,,,,,,,,
ARKH3b,PI-*314,,1:𐙉 ,2:𐙖 ,,,,3:𐙍 ,,,,,
HT100!,SA-RA₂,,2:𐙗 ,1:𐘸 5:𐜋 6:𐜗 7:𐜍 ,,,3:𐘝 ,4:𐙍 ,,,,,1:𐛧 1:𐛃 2:𐙙 2:𐛩 2:𐙂𐘁 
HT101,*79-*22F-DI,,1:𐛱 ,2:𐜋 3:𐜗 4:𐜓 ,,,,,,,,,
HT101,SA-RA₂,,1:𐙉 ,2:𐙖 ,,,,,,,,,3:𐚸 
HT101!,KU-PA₃,,,1:𐜋 3:𐜓 ,2:𐙘 ,4:𐙋 5:𐛺 ,,,,,,,
HT102,KA-PA,,1:𐙉 2:𐛭 3:𐛭 ,,,,,,,,,,
HT104,TA-PA,,,,,,,,,,,,1:𐚷 
HT114a,SA-RA₂,,1:𐙉 ,2:𐙖 ,,,3:𐘝 ,4:𐙍 ,,,,5:𐘖 ,
HT115b,TI-NU-JA,,1:𐙉 ,,,,,,,,,,
HT115b,KU-RU-MA,,,,,,,,,,,,1:𐘚 
HT116a,KU-PA-JA,,1:𐙉 ,,,,,,,,,,
HT116a,PU-RA₂,,1:𐙉 ,2:𐜉 ,,,,,,,,,
HT116a,SI+SE,,1:𐙉 ,2:𐜓 3:𐜗 ,,4:𐙋 ,,,,,,,
HT116a,PI-*34-TE,,1:𐙉 ,2:𐜗 ,,3:𐛺 ,,,,,,,
HT116a,SI-KI-NE,,1:𐙉 ,,2:𐙘 ,,,,,,,,
HT116a,QA-NU-MA,,1:𐙉 ,,2:𐙘 ,,,,,,,,
HT12,QA-TI-DA-TE,,,1:𐜉 ,2:𐙘 ,,,,,,,,
HT12,QE-TU-NE,,,,,,,,1:𐚻 ,2:𐘡 ,,,
HT12,*560,1:𐙈 ,2:𐜚 ,,,,3:𐘝 ,,4:𐚼 ,,,,5:𐘀𐘚 


In [494]:
# Print a table showing the number of times a commodity group is preceded by another commodity group
def groupForCommodity(word):
    for index, commodities in enumerate(commodity_order):
        if word in commodities:
            return index
    return -1

def collectCommodityAdjacency(word_infos, inscription, siblings):
    pc = ""
    for word_data in word_infos:
        if "description" not in word_data:
            continue
        if word_data["description"] == "recipient":
            pc = ""
            continue
        if word_data["description"] != "commodity":
            continue
        c = word_data["word"].replace('\U0001076b', '')
        i = groupForCommodity(c)
        j = groupForCommodity(pc)
        pc = c
        if i == j or -1 in [i, j]:
            continue
        siblings[i][j] += 1
    return siblings



In [495]:
n = len(commodity_order)
left_siblings = [[0] * n for i in range(n)]
for inscription in inscriptions:
    site = sites[inscription["name"]]
    if site != "Haghia Triada":
        continue
    if inscription["name"] not in has_commodity_order:
        continue
    word_infos = inscription["words"]
    left_siblings = collectCommodityAdjacency(word_infos, inscription, left_siblings)

group_names = ["bronze", "grain", "oil", "coriander", "olive",
                "figs", "wine", "QA", "E", "MI", "livestock"]

heading = ("inscription, recipient, bronze, grain, oil, coriander, olive,"
            "figs, wine, QA, E, MI, livestock, uncategorized")
print("," + ','.join(group_names))
for i, line in enumerate(left_siblings):
    print(group_names[i] + ',' + ','.join(str(x) if j != i else "N/A" for j, x in enumerate(line)))

,bronze,grain,oil,coriander,olive,figs,wine,QA,E,MI,livestock
bronze,N/A,0,0,0,0,0,0,0,0,0,0
grain,1,N/A,0,0,0,0,0,0,0,0,0
oil,0,16,N/A,3,0,2,2,0,0,1,1
coriander,0,6,9,N/A,0,0,0,0,0,0,0
olive,0,0,6,4,N/A,1,0,0,0,0,0
figs,0,11,5,2,2,N/A,0,0,0,0,0
wine,0,0,1,0,1,10,N/A,1,0,1,0
QA,0,1,2,0,0,1,1,N/A,2,0,0
E,0,1,2,0,0,0,1,4,N/A,0,0
MI,0,1,1,0,0,0,0,2,1,N/A,0
livestock,0,0,0,0,0,0,3,0,0,1,N/A


In [497]:
n = len(commodity_order)
left_siblings = [[0] * n for i in range(n)]
for inscription in inscriptions:
    site = sites[inscription["name"]]
    if site == "Haghia Triada":
        continue
    word_infos = inscription["words"]
    left_siblings = collectCommodityAdjacency(word_infos, inscription, left_siblings)

group_names = ["bronze", "grain", "oil", "coriander", "olive",
                "figs", "wine", "QA", "E", "MI", "livestock"]

heading = ("inscription, recipient, bronze, grain, oil, coriander, olive,"
            "figs, wine, QA, E, MI, livestock, uncategorized")
print("," + ','.join(group_names))
for i, line in enumerate(left_siblings):
    print(group_names[i] + ',' + ','.join(str(x) if j != i else "N/A" for j, x in enumerate(line)))

,bronze,grain,oil,coriander,olive,figs,wine,QA,E,MI,livestock
bronze,N/A,0,0,0,0,0,0,0,0,0,0
grain,0,N/A,1,1,0,1,1,0,0,0,0
oil,0,1,N/A,0,0,0,0,0,0,0,0
coriander,0,1,0,N/A,0,0,0,0,0,0,0
olive,0,0,0,0,N/A,0,0,0,0,0,0
figs,0,4,1,0,0,N/A,0,0,0,0,0
wine,0,3,1,0,0,1,N/A,0,0,0,0
QA,0,0,0,0,0,0,0,N/A,0,0,0
E,0,0,0,0,0,0,0,0,N/A,0,0
MI,0,0,0,0,0,0,0,0,0,N/A,0
livestock,0,0,0,0,0,0,0,0,0,0,N/A
